In [1]:
import os
os.chdir('/Users/furuku/Documents/thesis/2020 Thesis - Thailand Schdeule 2005-2019')
os.getcwd()

'/Users/furuku/Documents/thesis/2020 Thesis - Thailand Schdeule 2005-2019'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.offline as pyo
import plotly.graph_objs as go
import json
import collections
%matplotlib inline

In [3]:
def make_data(y2019,country,thai,x):
    data_airport = country.append(thai)
    region = data_airport[['Code', 'Global Region','Country Name']]
    # region = region.rename(columns={"IATA Metro": "Orig"})
#     print(region.head(10))
    region = region.drop_duplicates()
    region = region.set_index('Code')
    region = region.dropna()
    
    y2019 = y2019[(y2019['Date'] == x) ]
#     analyze = pd.merge(y2019, region, left_on=['Orig'], right_index=True)
    analyze = y2019.join(region, on=['Orig'])
    analyze = analyze.rename(columns={"Global Region": "Region_orig", "Country Name": "country_orig"})
    analyze = analyze.join(region, on=['Dest'])
    analyze = analyze.rename(columns={"Global Region": "Region_dest", "Country Name": "country_dest"})
#     print(analyze.shape)
#     print(analyze[['Orig']].head(10))
    lis = []
    
    for index,row in analyze.iterrows():
        tmp = str(row['Arr Time'])
        if len(tmp) < 4 :
            add = 4 - len(tmp)
            tmp = ("0" * add) + tmp
        if tmp == '2400':
            tmp = '2359'

        st = str(x) +' '+ tmp[:-2] + ':' +  tmp[-2:] + ':00' 
        lis.append(st)
        
    analyze['time'] = lis
    analyze['time'] = pd.to_datetime(analyze['time'] )
    
    arrive = list(y2019.Orig.unique())
#     arrive.remove('BKK')
    test = pd.DataFrame(arrive)
    test = test.join(region, on=0).rename(columns = {0:'station'})
    
    
    return analyze , test
    
    

In [4]:
def combine_hour(analyze,x,choose,check):
    
    analyze = analyze[analyze[choose] == 'BKK']
    
    if choose == 'Dest':
        if check:
            plo = analyze.resample(x, on= "time").agg({'Date' : 'count',
                                      'Orig' : lambda x: list(x),
                                      'Op Al' : lambda x: list(x),
                                      'Region_orig' : lambda x: list(x),
                                       'Alliance' : lambda x: list(x),
                                       'country_orig' : lambda x: list(x),
                                       'Seats' : lambda x: list(x)  })
        else:
            plo = analyze.resample(x, on= "time").agg({'Date' : 'count',
                                      'Orig' : lambda x: list(set(x)),
                                      'Op Al' : lambda x: list(set(x)),
                                      'Region_orig' : lambda x: list(set(x)),
                                       'Alliance' : lambda x: list(set(x)),
                                       'country_orig' : lambda x: list(set(x)),
                                       'Seats' : lambda x: list(x)})
            
    else:
        if check:
            plo = analyze.resample(x, on= "time").agg({'Date' : 'count',
                                      'Dest' : lambda x: list(x),
                                      'Op Al' : lambda x: list(x),
                                      'Region_dest' : lambda x: list(x),
                                       'Alliance' : lambda x: list(x),
                                       'country_dest' : lambda x: list(x),
                                       'Seats' : lambda x: list(x)  })
        else:
            plo = analyze.resample(x, on= "time").agg({'Date' : 'count',
                                      'Dest' : lambda x: list(set(x)),
                                      'Op Al' : lambda x: list(set(x)),
                                      'Region_dest' : lambda x: list(set(x)),
                                       'Alliance' : lambda x: list(set(x)),
                                       'country_dest' : lambda x: list(set(x)),
                                       'Seats' : lambda x: list(x)})
        
       


        
    return plo

In [5]:
def freq(plo,choose,check):
    
    if(check):
        dic = {}
        for index,row in plo.iterrows():
            key = list(row[choose])
            val = [0] * len(key)
            seat = dict(zip(key, val))

            my_list = list(row[choose])
            val2 = list(row['Seats'])
            for i in range(len(my_list)):
                seat[my_list[i]] = seat[my_list[i]] + val2[i]

            dic[row['time']] = seat
    
    else:

        dic = {}
        for index,row in plo.iterrows():
            my_list = list(row[choose])
            ctr = collections.Counter(my_list)
            tmp = dict(ctr)
            dic[row['time']] = tmp

    
    return dic

In [6]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))

In [7]:
def check_in(plo):
    plo[['Orig']] = len(plo[['Orig']])
    return plo

In [8]:
def every_time(plo):
    plo = plo.groupby('time').agg({'Date' : 'count',
                                  'Orig' : lambda x: list(x),
                                  'Op Al' : lambda x: list(x),
                                  'Global Region' : lambda x: list(x),
                                   'Alliance' : lambda x: list(x),
                                   'Country Name' : lambda x: list(x)})
    plo = plo.reset_index()
    
    return plo

In [9]:
def sum_seat(plo,func,x,analyze):
    
    key = list(analyze[x].unique())
    val = [0] * len(key)
    seat = dict(zip(key, val))
    
    if(func == True):
        for index,row in plo.iterrows():
            for i in range(len(row[x])):
                seat[row[x][i]] = seat[row[x][i]] + row['Seats'][i]
#                 seat[row['Global Region'][i]] = seat[row['Global Region'][i]] + 1
    else:
        for index,row in plo.iterrows():
            for i in range(len(row[x])):
#                 seat[row['Global Region'][i]] = seat[row['Global Region'][i]] + row['Seats'][i]
                seat[row[x][i]] = seat[row[x][i]] + 1
    
    trace =go.Pie(labels=list(seat.keys()),
              values=list(seat.values()),
              hoverinfo='label+percent',
              textinfo='value'
             )
    data = [trace]

    layout = go.Layout(title='จำนวนที่นั่งในการบินไปแต่ละทวีป')

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    return seat
    
        

In [10]:
y2019 = pd.read_excel("2019_Schedule_Daily_Extract_Report_8030.xlsx")

In [11]:
y2019.shape

(772842, 26)

In [12]:
# y2019 = y2019[(y2019.Orig == 'BKK')  |  (y2019.Dest == 'BKK')]
# y2019 = y2019[(y2019.Dest == 'BKK')]

In [13]:
y2019.shape

(772842, 26)

In [14]:
country = pd.read_excel("1.-Airport_Lookup_Country-Region.xlsx")

In [15]:
thai = pd.read_excel("Thailand_Airport_Lookup.xlsx")

In [16]:
y2019

,Date,Mkt Al,Alliance,Op Al,Orig,Dest,Kilometers,Flight,Sequence,Routing,...,Econ,Other,Dep Term,Arr Term,Dep Time,Arr Time,Block Mins,Arr Flag,Orig WAC,Dest WAC
0,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,512,1,BKK-SIN,...,0,0,NaN,1,920,1255,155,0,782,776
1,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,520,1,BKK-SIN,...,0,0,NaN,1,1150,1525,155,0,782,776
2,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,516,1,BKK-SIN,...,0,0,NaN,1,1335,1705,150,0,782,776
3,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,514,1,BKK-SIN,...,0,0,NaN,1,2120,50,150,1,782,776
4,2019-01-01,3K,Low Cost Carriers,3K,HKT,SIN,976.871811,534,1,HKT-SIN,...,0,0,D,1,935,1230,115,0,782,776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772837,2019-12-31,ZH,Star Alliance,ZH,HKT,SZX,2311.017992,9016,1,HKT-SZX,...,159,0,I,NaN,10,500,230,0,782,713
772838,2019-12-31,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9003,1,SZX-BKK,...,153,0,NaN,NaN,1155,1405,190,0,713,782
772839,2019-12-31,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9001,1,SZX-BKK,...,159,0,NaN,NaN,1620,1830,190,0,713,782
772840,2019-12-31,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9019,1,SZX-BKK,...,153,0,NaN,NaN,2245,125,220,1,713,782


In [17]:
analyze, test = make_data(y2019,country,thai,'2019-03-04')
analyze

tmp = analyze.copy()


In [18]:

from nested_dict import nested_dict

table = nested_dict(2, int)
table[1] = '1234'
table[2][3] = '5555'

table

nested_dict(<function nested_dict.implementation._nested_levels.<locals>.<lambda>()>,
            {1: '1234', 2: _recursive_dict(int, {3: '5555'})})

In [19]:
analyze

,Date,Mkt Al,Alliance,Op Al,Orig,Dest,Kilometers,Flight,Sequence,Routing,...,Arr Time,Block Mins,Arr Flag,Orig WAC,Dest WAC,Region_orig,country_orig,Region_dest,country_dest,time
139090,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,512,1,BKK-SIN,...,1255,155,0,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 12:55:00
139091,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,516,1,BKK-SIN,...,1705,150,0,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 17:05:00
139092,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,510,1,BKK-SIN,...,1830,155,0,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 18:30:00
139093,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,518,1,BKK-SIN,...,15,155,1,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 00:15:00
139094,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,514,1,BKK-SIN,...,50,150,1,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 00:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141289,2019-03-04,ZH,Star Alliance,ZH,JJN,BKK,2241.816200,8011,2,WUX-JJN-BKK,...,1515,195,0,713,782,Asia,China,Asia,Thailand,2019-03-04 15:15:00
141290,2019-03-04,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9003,1,SZX-BKK,...,1405,175,0,713,782,Asia,China,Asia,Thailand,2019-03-04 14:05:00
141291,2019-03-04,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9001,1,SZX-BKK,...,1830,185,0,713,782,Asia,China,Asia,Thailand,2019-03-04 18:30:00
141292,2019-03-04,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9019,1,SZX-BKK,...,115,180,1,713,782,Asia,China,Asia,Thailand,2019-03-04 01:15:00


In [20]:
group_country = {}
group_region = {}
country = list(test['Country Name'].unique())
reg = list(test['Global Region'].unique())

for coun in country:
    group_country[coun] = list(test[test['Country Name'] == coun]['station'])


for coun in reg:
    group_region[coun] = list(test[test['Global Region'] == coun]['station'])

In [21]:
# analyze = tmp[tmp.Orig.isin(group_region['Middle East'])]
# analyze = tmp[tmp.Orig.isin(group_region['Russian Federation'])]
# analyze = tmp[tmp.Orig.isin(group_region['Europe'])]
# analyze = tmp[tmp.Orig.isin(group_country['Thailand'])]
# analyze = tmp[tmp.Orig.isin((Diff(group_region['Asia'], group_country['Thailand'])))]
# analyze = tmp[tmp.Orig.isin(['HKT', 'CNX', 'HKG', 'ICN', 'SIN', 'USM', 'TPE', 'PVG', 'CAN', 'DEL', 'KBV', 'HND', 'PEK', 'MNL', 'PNH', 'KUL', 'CEI', 'SGN', 'RGN', 'LHR', 'BOM', 'NRT', 'PUS', 'HAN', 'KMG', 'DXB', 'KKC', 'CTU', 'FRA', 'REP', 'VTE', 'KIX', 'CCU', 'UTH', 'SYD', 'HDY', 'CDG', 'NGO', 'DAC', 'MEL', 'CGK', 'ZRH', 'CMB', 'SZX'])]
# analyze = tmp[tmp.Orig.isin(['VIE', 'HYD', 'TLV', 'UBP', 'WUH', 'MCT', 'MXP', 'AMS', 'MAA', 'SVO', 'CPH', 'XMN', 'DPS', 'KTM', 'CTS', 'LPQ', 'ARN', 'LHE', 'OSL', 'MUC', 'HGH', 'MDL', 'OVB', 'DOH', 'CGO', 'DAD', 'URT', 'HEL', 'PER', 'KHH', 'FUK', 'KHI', 'NNG', 'AKL', 'PEN', 'TDX', 'BRU', 'TSN', 'VNS', 'NGB', 'AUH', 'ATQ', 'SHE'])]
# analyze = tmp[tmp.Orig.isin(['KWI', 'MFM', 'AMD', 'LPT', 'HAK', 'PBH', 'FCO', 'SWA', 'SJW', 'GAY', 'NKG', 'CJU', 'OKA', 'ISB', 'CKG', 'IKT', 'BNE', 'TNA', 'TAO', 'THS', 'TAE', 'JAI', 'IST', 'NAW', 'LHW', 'KWE', 'YTY', 'IKA', 'DME', 'KHN', 'MLE', 'HFE', 'DLI', 'RMQ', 'KJA', 'DUS', 'BAH', 'FOC', 'ALA', 'MWX', 'PQC', 'LKO', 'TYN'])]
# analyze = tmp[tmp.Orig.isin(['BWN', 'URC', 'ADD', 'VVO', 'CSX', 'JJN', 'XIY', 'KBP', 'NBO', 'HPH', 'WNZ', 'RUN', 'WUS', 'CEB', 'AMM', 'ULN', 'CAI', 'ASB', 'CGP', 'LYG', 'HRB', 'CGQ', 'TAS', 'DYG', 'TSE', 'LYA', 'LED', 'BHY', 'CEK', 'KUF', 'PKZ', 'PEE', 'BQS', 'NYT', 'KHV', 'UFA', 'YNT', 'XNN', 'DSN', 'CXR', 'GAU', 'IXB'])]
analyze



,Date,Mkt Al,Alliance,Op Al,Orig,Dest,Kilometers,Flight,Sequence,Routing,...,Arr Time,Block Mins,Arr Flag,Orig WAC,Dest WAC,Region_orig,country_orig,Region_dest,country_dest,time
139090,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,512,1,BKK-SIN,...,1255,155,0,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 12:55:00
139091,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,516,1,BKK-SIN,...,1705,150,0,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 17:05:00
139092,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,510,1,BKK-SIN,...,1830,155,0,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 18:30:00
139093,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,518,1,BKK-SIN,...,15,155,1,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 00:15:00
139094,2019-03-04,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,514,1,BKK-SIN,...,50,150,1,782,776,Asia,Thailand,Asia,Singapore,2019-03-04 00:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141289,2019-03-04,ZH,Star Alliance,ZH,JJN,BKK,2241.816200,8011,2,WUX-JJN-BKK,...,1515,195,0,713,782,Asia,China,Asia,Thailand,2019-03-04 15:15:00
141290,2019-03-04,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9003,1,SZX-BKK,...,1405,175,0,713,782,Asia,China,Asia,Thailand,2019-03-04 14:05:00
141291,2019-03-04,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9001,1,SZX-BKK,...,1830,185,0,713,782,Asia,China,Asia,Thailand,2019-03-04 18:30:00
141292,2019-03-04,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9019,1,SZX-BKK,...,115,180,1,713,782,Asia,China,Asia,Thailand,2019-03-04 01:15:00


In [22]:
plo = combine_hour(analyze,'15min','Dest',False)
plo = plo.reset_index()
plo['sum_seat'] = plo['Seats'].map(sum)
plo

# plo

,time,Date,Orig,Op Al,Region_orig,Alliance,country_orig,Seats,sum_seat
0,2019-03-04 00:00:00,2,"[HKG, ICN]","[TW, CX]",[Asia],"[oneworld, Low Cost Carriers]","[South Korea, Hong Kong]","[262, 189]",451
1,2019-03-04 00:15:00,4,"[ICN, PUS, DYG]","[7C, KE, OZ, CZ]",[Asia],"[None, SkyTeam Alliance, Low Cost Carriers, St...","[South Korea, China]","[151, 276, 290, 189]",906
2,2019-03-04 00:30:00,6,"[TSN, OKA, PUS, HKT, AMD, HKG]","[MM, VZ, ET, SG, KE, CA]",[Asia],"[SkyTeam Alliance, Low Cost Carriers, Star All...","[South Korea, Hong Kong, China, Japan, Thailan...","[276, 180, 270, 180, 167, 200]",1273
3,2019-03-04 00:45:00,4,"[KMG, CNX, PVG, ICN]","[7C, MU, VZ]",[Asia],"[SkyTeam Alliance, Low Cost Carriers]","[South Korea, China, Thailand]","[168, 189, 180, 292]",829
4,2019-03-04 01:00:00,7,"[CJU, WUH, CAN, PVG, CCU, XIY, MLE]","[CZ, 6E, Q2, 9C, ZE, BK]",[Asia],"[None, Low Cost Carriers]","[South Korea, China, India, Maldives]","[186, 160, 180, 149, 186, 173, 151]",1185
...,...,...,...,...,...,...,...,...,...
91,2019-03-04 22:45:00,7,"[CAN, KBV, NRT, HKT, CSX, ICN, NYT]","[VZ, TG, PG, KE, NH, WE]",[Asia],"[None, SkyTeam Alliance, Low Cost Carriers, St...","[South Korea, China, Myanmar, Japan, Thailand]","[321, 180, 168, 70, 299, 407, 215]",1660
92,2019-03-04 23:00:00,6,"[USM, HDY, TPE, CNX, KUL, HKG]","[PG, HX, WE, MH, TG]",[Asia],"[None, oneworld, Star Alliance]","[Thailand, Hong Kong, Malaysia, Taiwan]","[162, 285, 120, 364, 160, 168]",1259
93,2019-03-04 23:15:00,2,"[KUL, HKG]","[EK, RJ]",[Asia],"[None, oneworld]","[Hong Kong, Malaysia]","[400, 270]",670
94,2019-03-04 23:30:00,7,"[CAN, PUS, NRT, HKT, TPE, CNX, HKG]","[VZ, LJ, PG, JL, CX, BR, KQ]",[Asia],"[Star Alliance, None, SkyTeam Alliance, onewor...","[South Korea, Hong Kong, China, Taiwan, Japan,...","[162, 206, 234, 180, 309, 334, 189]",1614


In [23]:
dic = freq(plo,'Orig',True)
dic

{Timestamp('2019-03-04 00:00:00'): {'HKG': 262, 'ICN': 189},
 Timestamp('2019-03-04 00:15:00'): {'ICN': 151, 'PUS': 276, 'DYG': 290},
 Timestamp('2019-03-04 00:30:00'): {'TSN': 276,
  'OKA': 180,
  'PUS': 270,
  'HKT': 180,
  'AMD': 167,
  'HKG': 200},
 Timestamp('2019-03-04 00:45:00'): {'KMG': 168,
  'CNX': 189,
  'PVG': 180,
  'ICN': 292},
 Timestamp('2019-03-04 01:00:00'): {'CJU': 186,
  'WUH': 160,
  'CAN': 180,
  'PVG': 149,
  'CCU': 186,
  'XIY': 173,
  'MLE': 151},
 Timestamp('2019-03-04 01:15:00'): {'NGB': 156,
  'YTY': 161,
  'PVG': 149,
  'TPE': 149,
  'ICN': 155,
  'NKG': 364,
  'TAO': 323,
  'SZX': 172},
 Timestamp('2019-03-04 01:30:00'): {'SYD': 165,
  'TNA': 189,
  'PVG': 400,
  'ICN': 156,
  'SHE': 172,
  'TAE': 186},
 Timestamp('2019-03-04 01:45:00'): {'MAA': 168, 'TYN': 180, 'PVG': 160},
 Timestamp('2019-03-04 02:00:00'): {'CAN': 161, 'CTU': 156, 'KMG': 174},
 Timestamp('2019-03-04 02:15:00'): {'TPE': 307, 'HKG': 334},
 Timestamp('2019-03-04 02:30:00'): {'CGO': 164, 'O

In [ ]:
# percent_seat = plo[['time','Date','sum_seat']]
# percent_seat['sum_seat'] = percent_seat['sum_seat'] / percent_seat['sum_seat'].sum() *100
# percent_seat.drop(percent_seat[percent_seat.sum_seat == 0 ].index)


In [ ]:
# percent_seat.sum()

In [ ]:
# ax = sns.barplot(x="time", y="sum_seat", data=percent_seat)

In [ ]:
# long = check_in(plo)
# long

In [ ]:
# len(list((plo[['Orig']].iloc[0]))[0])
# plo['Global Region'].str.len()

In [ ]:
# dic = {}
# for index,row in plo.iterrows():
#     key = list(plo['Orig'].unique())
#     val = [0] * len(key)
#     seat = dict(zip(key, val))
        
#     my_list = list(row[choose])
#     val = list(row['Seats'])
#     for i in range(len(my_list)):
#         seat[my_list[i]] = seat[my_list[i]] + val[i]
            
#     dic[row['time']] = seat

In [24]:
#true is seat
dic = freq(plo,'Orig',True)

In [25]:
dic

{Timestamp('2019-03-04 00:00:00'): {'HKG': 262, 'ICN': 189},
 Timestamp('2019-03-04 00:15:00'): {'ICN': 151, 'PUS': 276, 'DYG': 290},
 Timestamp('2019-03-04 00:30:00'): {'TSN': 276,
  'OKA': 180,
  'PUS': 270,
  'HKT': 180,
  'AMD': 167,
  'HKG': 200},
 Timestamp('2019-03-04 00:45:00'): {'KMG': 168,
  'CNX': 189,
  'PVG': 180,
  'ICN': 292},
 Timestamp('2019-03-04 01:00:00'): {'CJU': 186,
  'WUH': 160,
  'CAN': 180,
  'PVG': 149,
  'CCU': 186,
  'XIY': 173,
  'MLE': 151},
 Timestamp('2019-03-04 01:15:00'): {'NGB': 156,
  'YTY': 161,
  'PVG': 149,
  'TPE': 149,
  'ICN': 155,
  'NKG': 364,
  'TAO': 323,
  'SZX': 172},
 Timestamp('2019-03-04 01:30:00'): {'SYD': 165,
  'TNA': 189,
  'PVG': 400,
  'ICN': 156,
  'SHE': 172,
  'TAE': 186},
 Timestamp('2019-03-04 01:45:00'): {'MAA': 168, 'TYN': 180, 'PVG': 160},
 Timestamp('2019-03-04 02:00:00'): {'CAN': 161, 'CTU': 156, 'KMG': 174},
 Timestamp('2019-03-04 02:15:00'): {'TPE': 307, 'HKG': 334},
 Timestamp('2019-03-04 02:30:00'): {'CGO': 164, 'O

In [26]:
plotly = pd.DataFrame(dic)
# plotly = plotly.reset_index()
plotly.T 

,HKG,ICN,PUS,DYG,TSN,OKA,HKT,AMD,KMG,CNX,...,DSN,BAH,BNE,MEL,AKL,PER,CSX,NYT,BHY,SYX
2019-03-04 00:00:00,262.0,189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-04 00:15:00,NaN,151.0,276.0,290.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-04 00:30:00,200.0,NaN,270.0,NaN,276.0,180.0,180.0,167.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-04 00:45:00,NaN,292.0,NaN,NaN,NaN,NaN,NaN,NaN,168.0,189.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-04 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-04 22:45:00,NaN,407.0,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,299.0,215.0,NaN,NaN
2019-03-04 23:00:00,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,364.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-04 23:15:00,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-04 23:30:00,189.0,NaN,206.0,NaN,NaN,NaN,180.0,NaN,NaN,334.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
summ = pd.DataFrame(plotly.sum(axis=1), columns = {'count'})
# type(summ)
# sorted(plotly.items())
summ.sort_values(by='count', ascending=False)
summ.iloc[1]
# sum

count    3480.0
Name: ICN, dtype: float64

In [28]:
plotly = pd.DataFrame(dic)
# plotly = plotly.reset_index()
# plotly = plotly.loc[['TG', 'PG', 'WE', 'VZ', 'CZ', 'MU', 'CX']]
plotly

,2019-03-04 00:00:00,2019-03-04 00:15:00,2019-03-04 00:30:00,2019-03-04 00:45:00,2019-03-04 01:00:00,2019-03-04 01:15:00,2019-03-04 01:30:00,2019-03-04 01:45:00,2019-03-04 02:00:00,2019-03-04 02:15:00,...,2019-03-04 21:30:00,2019-03-04 21:45:00,2019-03-04 22:00:00,2019-03-04 22:15:00,2019-03-04 22:30:00,2019-03-04 22:45:00,2019-03-04 23:00:00,2019-03-04 23:15:00,2019-03-04 23:30:00,2019-03-04 23:45:00
HKG,262.0,NaN,200.0,NaN,NaN,NaN,NaN,NaN,NaN,334.0,...,NaN,NaN,309.0,NaN,309.0,NaN,168.0,270.0,189.0,NaN
ICN,189.0,151.0,NaN,292.0,NaN,155.0,156.0,NaN,NaN,NaN,...,144.0,NaN,NaN,70.0,NaN,407.0,NaN,NaN,NaN,240.0
PUS,NaN,276.0,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,292.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN
DYG,NaN,290.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TSN,NaN,NaN,276.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,299.0,NaN,NaN,NaN,NaN,NaN,NaN
CSX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,299.0,NaN,NaN,NaN,NaN
NYT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,215.0,NaN,NaN,NaN,NaN
BHY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189.0


In [29]:
# plotly.T.plot(kind='bar', stacked=True)

In [30]:
data = []
count = 0
for index, row in plotly.iterrows():
    trace =go.Bar(x=plotly.T.index,
              y=plotly.iloc[count],
#               marker={'color':'#9ACD32'},
                   text=plotly.index[count],
              name=plotly.index[count])
    data.append(trace)
    count = count+1
#     if count == 24:
#         break
    
layout = go.Layout(title='ANALYZE BANK',
                   xaxis = {'title':'Region'},
                   yaxis = {'title':'Population'}, 
                   hovermode='closest',
                   barmode='stack')
# fig.update_layout(barmode="relative")
fig = go.Figure(data=data,layout=layout)
fig.update_traces(texttemplate= '%{text}', textposition='inside')

# fig.show()
pyo.plot(fig)

'temp-plot.html'

In [ ]:
# data = []
# count = 0
# for index, row in plotly.iterrows():
#     trace =go.Bar(x=plotly.T.index,
#               y=plotly.iloc[count],
# #               marker={'color':'#9ACD32'},
                 
#               name=plotly.index[count])
#     data.append(trace)
#     count = count+1
# #     if count == 24:
# #         break
    
# layout = go.Layout(title='Thailand Population',
#                    xaxis = {'title':'Region'},
#                    yaxis = {'title':'Population'}, 
#                    hovermode='closest',
#                    barmode='stack')
# # fig.update_layout(barmode="relative")
# fig = go.Figure(data=data,layout=layout)

# fig.show()

In [ ]:
plotly.T

In [ ]:
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


trace1 = go.Scatter(x=plo.time,
                    y=plo.Date,
                    name = "plotly example",
                    line = dict(color = 'blue'),
                    opacity = 0.4)

layout = dict(title='plotly example',)

fig = dict(data=[trace1], layout=layout)
iplot(fig)

In [ ]:
trace1 = go.Scatter(x=plo.time,
                    y=plo.sum_seat,
                    name = "plotly example",
                    line = dict(color = 'blue'),
                    opacity = 0.4)

layout = dict(title='plotly example',)

fig = dict(data=[trace1], layout=layout)
iplot(fig)

In [ ]:
plo

In [ ]:
# true == cum_seat
seat = sum_seat(plo,True,'Region_orig',analyze)

In [ ]:
seat

In [ ]:
df = pd.DataFrame(seat.values(),index = seat.keys(), columns = {'time'})
df['time'] = df['time'] / df['time'].sum() * 100
df = df.sort_values('time', ascending=False).head(10)
df

In [ ]:
chart = pd.DataFrame()

In [ ]:
dest = combine_hour(analyze,'15min','Dest',False)
# dest = dest.reset_index()
dest['sum_seat'] = dest['Seats'].map(sum)
dest




In [ ]:
orig = combine_hour(analyze,'15min','Orig',False)
# orig = orig.reset_index()
orig['sum_seat'] = orig['Seats'].map(sum)
orig

In [ ]:
orig['sum_seat']

In [ ]:
chart['arr'] = dest['sum_seat']
chart['dep'] = orig['sum_seat']
cumulative=  chart['arr'] - chart['dep']
# lis = []
# for index, row in chart.iterrows():
    

cumu = 0
lis = []
for i in list(cumulative):
    cumu += i
    lis.append(cumu)
    
chart['cumulative'] = lis
chart

In [ ]:
chart['cumulative'].mean()

In [ ]:
# [str(i)  for i in list(chart.index)]
[i  for i in chart['arr']]
[i  for i in chart['dep']]
[i  for i in chart['cumulative']]
# [chart['cumulative'].mean()] * len([i  for i in chart['arr']])

In [ ]:
# chart[['time','cumulative']].to_dict('list')

In [ ]:
cumu = 0
lis = []
for i in list(chart['cumulative']):
    cumu += i
    lis.append(cumu)
    
chart['chart'] = lis
chart


In [ ]:
[chart.chart.mean()] * 96

In [ ]:
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


trace1 = go.Scatter(x=chart.time,
                    y=chart.chart,
                    name = "plotly example",
                    line = dict(color = 'blue'),
                    opacity = 0.4)

trace2 = go.Scatter(x=chart.time,
                    y=[chart.chart.mean()] * 96,
                    name = "plotly example",
                    line = dict(color = 'red'),
                    opacity = 0.4)

layout = dict(title='plotly example',)

fig = dict(data=[trace1,trace2], layout=layout)
iplot(fig)

In [ ]:
dest[['sum_seat']]

In [ ]:
y2019

In [ ]:
tmp = y2019.copy()
d = tmp[(tmp.Dest == 'BKK')]

In [ ]:
d.Dest.value_counts()

In [ ]:
d.Orig.value_counts()

In [ ]:
y2019[y2019.Orig == 'BKK']

In [ ]:
dic = freq(dest,'Orig',True)

In [ ]:
tmp = y2019.copy()
d = tmp[(tmp.Dest == 'BKK')]
d['Seats'].sum()

In [ ]:
pd.DataFrame(d.groupby('Orig')['Seats'].sum()/d['Seats'].sum()).sort_values('Seats', ascending=False).head(10).to_dict()['Seats']

In [ ]:
pd.DataFrame(d.groupby('Orig')['Seats'].count()).sort_values('Seats', ascending=False).head(10)

In [ ]:
pd.Series(df.A.values,index=df.B).to_dict()